# Tutorial 1: A basic avatarization

In this tutorial, we will connect to a server to perform the avatarization of a dataset that does not require any pre-processing. We'll retrieve the anonymized dataset and the associated avatarization report. 

## Connection

In [1]:
import os

url = os.environ.get("AVATAR_BASE_URL")
username = os.environ.get("AVATAR_USERNAME")
password = os.environ.get("AVATAR_PASSWORD")
username

'julien@octopize.io'

In [2]:
# This is the client that you'll be using for all of your requests
from avatars.client import ApiClient
from avatars.models import AvatarizationJobCreate, AvatarizationParameters
from avatars.models import ReportCreate

import pandas as pd
import io

# Change this to your actual server endpoint, e.g. base_url="https://avatar.company.com"
client = ApiClient(base_url=url)
client.authenticate(username=username, password=password)

In [3]:
# Verify that we can connect to the API server
client.health.get_health()

{'message': 'ok'}

In [4]:
# Verify that the client is compatible.
client.compatibility.is_client_compatible()

CompatibilityResponse(message='Compatible client.', most_recent_compatible_client='0.10.0', status=<CompatibilityStatus.compatible: 'compatible'>)

## Loading data

We recommend loading your csv file as a pandas dataframe. It enables you to check your data before avatarization and to pre-process it if required. 

In this tutorial, we use the simple and well-known `iris` dataset to demonstrate the main steps of an avatarization.

In [5]:
df = pd.read_csv("../fixtures/iris.csv")

In [6]:
df

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [7]:
dataset = client.pandas_integration.upload_dataframe(df)
print(dataset)

id=UUID('ede1736a-93b9-46aa-9580-fec047e8016f') hash='e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855' name=None columns=[ColumnDetail(type=<ColumnType.float: 'float'>, label='sepal.length', is_identifier=False), ColumnDetail(type=<ColumnType.float: 'float'>, label='sepal.width', is_identifier=False), ColumnDetail(type=<ColumnType.float: 'float'>, label='petal.length', is_identifier=False), ColumnDetail(type=<ColumnType.float: 'float'>, label='petal.width', is_identifier=False), ColumnDetail(type=<ColumnType.category: 'category'>, label='variety', is_identifier=False)] download_url='https://octopize.octopize.app/datasets/ede1736a-93b9-46aa-9580-fec047e8016f/download' analysis_status=None analysis_duration=None nb_lines=150 nb_dimensions=7 summary=None filetype=<FileType.parquet: 'parquet'>


The data has now been loaded onto the server. 

Note that it is also possible to directly load a csv file without using pandas. 

In [8]:
filename = "../fixtures/iris.csv"
import time

with open(filename, "r") as f:

    dataset = client.datasets.create_dataset(request=f)
print(dataset)

id=UUID('3f16a425-0335-4a4f-806c-82dfe9f1c62b') hash='e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855' name=None columns=[ColumnDetail(type=<ColumnType.float: 'float'>, label='sepal.length', is_identifier=False), ColumnDetail(type=<ColumnType.float: 'float'>, label='sepal.width', is_identifier=False), ColumnDetail(type=<ColumnType.float: 'float'>, label='petal.length', is_identifier=False), ColumnDetail(type=<ColumnType.float: 'float'>, label='petal.width', is_identifier=False), ColumnDetail(type=<ColumnType.category: 'category'>, label='variety', is_identifier=False)] download_url='https://octopize.octopize.app/datasets/3f16a425-0335-4a4f-806c-82dfe9f1c62b/download' analysis_status=None analysis_duration=None nb_lines=150 nb_dimensions=7 summary=None filetype=<FileType.csv: 'csv'>


## Analyze your data

A tool to analyze the data prior to an avatarization is provided. It computes several statistics that can be useful to:
- confirm that the data loaded is as expected and
- give insight on potential transformation to the data that are required (this will be covered in later tutorials)

In [ ]:
dataset

In [ ]:
import time
from avatars.models import AnalysisStatus

dataset = client.datasets.analyze_dataset(dataset.id)

while dataset.analysis_status != AnalysisStatus.done:
    dataset = client.datasets.get_dataset(dataset.id)
    time.sleep(1)

print(f"Lines: {dataset.nb_lines}, dimensions: {dataset.nb_dimensions}")

In [ ]:
print(dataset.summary)

In [ ]:
for var in dataset.summary.stats:
    print("---------")
    for stat in var:
        print(stat)

## Creating and launching an avatarization job

In [ ]:
avatarization_job = client.jobs.create_avatarization_job(
    AvatarizationJobCreate(
        parameters=AvatarizationParameters(k=20, dataset_id=dataset.id),
    )
)

## Retrieving the completed avatarization job

In [ ]:
avatarization_job = client.jobs.get_avatarization_job(avatarization_job.id, timeout=100)
print(avatarization_job.status)
print(avatarization_job.result)  # there is no metrics

## Retrieving the avatars

In [ ]:
# Download the avatars as a string
avatars_str = client.datasets.download_dataset(avatarization_job.result.avatars_dataset.id)

# Download the avatars as a pandas dataframe
avatars_df = client.pandas_integration.download_dataframe(avatarization_job.result.avatars_dataset.id)

In [ ]:
print(avatars_str)

In [ ]:
print(avatars_df)

## Creating and launching a privacy metrics job

In [ ]:
from avatars.models import PrivacyMetricsJobCreate, PrivacyMetricsParameters

privacy_job = client.jobs.create_privacy_metrics_job(
    PrivacyMetricsJobCreate(
        parameters=PrivacyMetricsParameters(
            original_id=dataset.id,
            unshuffled_avatars_id=avatarization_job.result.sensitive_unshuffled_avatars_datasets.id,
            closest_rate_percentage_threshold=0.3,
            closest_rate_ratio_threshold=0.3,
            known_variables=[
                "sepal.length",
                "petal.length",
            ],
            target="variety",
            seed=42,
        ),
    )
)

privacy_job = client.jobs.get_privacy_metrics(privacy_job.id, timeout=100)

print(privacy_job.status)
print(privacy_job.result)

In [ ]:
privacy_metrics = privacy_job.result
print("*** Privacy metrics ***")
for metric in privacy_metrics:
    print(metric)

## Creating and launching a signal metrics job

In [ ]:
from avatars.models import SignalMetricsJobCreate, SignalMetricsParameters

signal_job = client.jobs.create_signal_metrics_job(
    SignalMetricsJobCreate(
        parameters=SignalMetricsParameters(
            original_id=dataset.id,
            avatars_id=avatarization_job.result.avatars_dataset.id,
            seed=42,
        ),
    )
)

signal_job = client.jobs.get_signal_metrics(signal_job.id, timeout=100)
print(signal_job.status)
print(signal_job.result)

In [ ]:
utility_metrics = signal_job.result
print("*** Utility metrics ***")
for metric in utility_metrics:
    print(metric)

## Retrieving the avatarization report

In [ ]:
report = client.reports.create_report(
    ReportCreate(
        avatarization_job_id=avatarization_job.id,
        privacy_job_id=privacy_job.id,
        signal_job_id=signal_job.id,
    ),
    timeout=30,
)
result = client.reports.download_report(id=report.id)

with open("./my_avatarization_report.pdf", "wb") as f:
    f.write(result)

The report is now generated and available on your machine

# How to print an error message 
There are multiple types of error and we encourage you to have a look at our [documentation](https://python.docs.octopize.io/latest/user_guide.html#understanding-errors) to understand them.

The most common error is when server validation prevents a job from running.

The following section show how to print an error message. 

In [ ]:
wrong_parameters = AvatarizationParameters(
    k=500, dataset_id=dataset.id
)  # k is too big (bigger than the dataset !)

avatarization_job = client.jobs.create_avatarization_job(
    AvatarizationJobCreate(
        parameters=wrong_parameters,
    )
)

avatarization_job = client.jobs.get_avatarization_job(avatarization_job.id, timeout=100)
print(avatarization_job.status)
print("----")
print(avatarization_job.error_message)

*In the next tutorial, we will show how to parameterize an avatarization.*